Créé le 17 mars 2021

**Projet Tableau de Bord** 

**Groupe n°3 - Arnaques en ligne**

**Insertion dans la base de données**

@authors:
- VEDIS Theo



#### Import libraries

In [ ]:
########## Module import ##########

# Fichiers
import json
from tqdm.notebook import tqdm

# Format
import datetime

# Base de données 
import Alimentation_BD as ABD


In [2]:
# recuperation de l'adresse de la BD
adresseBD : str = "oracle://" + username + ":" + password + "@telline.univ-tlse3.fr:1521/etupre"

#### Important des données 

In [ ]:
# Load data
with open(r"C:\Users\Stior\Documents\cours\M1SID\S2\Dashboard\code\insertion_articles.json", "r", encoding="utf-8") as f:
    data : dict = json.load(f)

#### Description des colonnes

In [ ]:
# visualisation des colonnes du dictionnaire
data["0"].keys()

dict_keys(['url', 'keyword', 'author', 'date', 'abstract', 'title', 'theme'])

In [ ]:
print("Title taille max :", max([len(row["title"]) for i, row in data.items()]))
print("Nom auteur taille max :", max([max([0] + list([len(j) for j in row["author"].keys()])) for i, row in data.items()]))
all_len : list = []
for i, row in data.items():
    for j, aff in row["author"].items():
        if len(aff) == 2:
            all_len.append(len(aff[0]))
print("Affiliation taille max :", max(all_len))

Title taille max : 279
Nom auteur taille max : 171
Affiliation taille max : 173


#### Insertion des données

In [ ]:
id_auteur : int = 0
for id, row in tqdm(data.items()):
    id_article : int = int(id)
    date  = datetime.datetime.strptime(row["date"], "%Y-%m-%d").strftime("%d/%m/%Y") # Format date

    if row["theme"] == "description des arnaques":
        theme = "description"
    else:
        theme = "detection" 

    title : str  = "".join([i for i in row["title"] if i.isascii()])

    retourVal, retour = ABD.insert_article(id_article, date, title, theme, adresseBD=adresseBD)
    if retourVal != 0 and retourVal != 1:
        print("Insert article:", retourVal, retour)

    for name, aff in row["author"].items():
        name : str  = "".join([i for i in name if i.isascii()])

        if len(aff) == 2:
            affiliation : str = "".join([i for i in aff[0] if i.isascii()])
            pays  : str = "".join([i for i in aff[1] if i.isascii()])

            if len(pays) > 60 :
                pays = None
        else:
            pays = None
            affiliation = None
        
        retourVal, retour = ABD.insert_auteur(id_auteur, name, affiliation, pays, adresseBD=adresseBD)
        if retourVal != 0 and retourVal != 1:
            print("Insert auteur:", retourVal, retour)

        retourVal, retour = ABD.insert_ecrire(id_article, id_auteur, adresseBD=adresseBD)
        if retourVal != 0 and retourVal != 1:
            print("Insert ecrire:", retourVal, retour)
        id_auteur += 1



Insert auteur: -12899.0 ORA-12899: valeur trop grande pour la colonne "RSV1589A"."AUTEUR"."NOM" (réelle : 139, maximum : 100)
Insert ecrire: -2291.0 ORA-02291: violation de contrainte d'intégrité (RSV1589A.FK_ECRIRE_AUTEUR) - clé parent introuvable
Insert auteur: -12899.0 ORA-12899: valeur trop grande pour la colonne "RSV1589A"."AUTEUR"."NOM" (réelle : 171, maximum : 100)
Insert ecrire: -2291.0 ORA-02291: violation de contrainte d'intégrité (RSV1589A.FK_ECRIRE_AUTEUR) - clé parent introuvable



#### Importation des types d'arnaques

In [ ]:
with open(r"C:\Users\Stior\Documents\cours\M1SID\S2\Dashboard\code\type_scams.json", "r", encoding="utf-8") as f:
    data : dict  = json.load(f)

#### Insertion des types d'arnaques


In [ ]:
id_arnaque : int = 0
for id, scam_type in tqdm(data.items()):
    id_article : int  = int(id)
    retourVal, retour = ABD.insert_arnaque(id_arnaque, scam_type, None, None, None, None, id_article, adresseBD)

    if retourVal != 0:
        print("Insert arnaque:", retourVal, retour)
    
    id_arnaque += 1
